# NQL Gridworld Pathfollowing

# Setup

In [1]:
%tensorflow_version 2.x
!test -d language_repo || git clone https://github.com/google-research/language.git language_repo
%cd /content/language_repo/language/nql
!pip install .

Cloning into 'language_repo'...
remote: Enumerating objects: 283, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (221/221), done.
remote: Total 1624 (delta 94), reused 175 (delta 59), pack-reused 1341
Receiving objects: 100% (1624/1624), 1.99 MiB | 5.94 MiB/s, done.
Resolving deltas: 100% (899/899), done.
/content/language_repo/language/nql
Processing /content/language_repo/language/nql
     |████████████████████████████████| 320.4MB 45kB/s 
  Created wheel for nql: filename=nql-0.0.1.dev0-cp36-none-any.whl size=48738 sha256=ef554fd242a328c0bd5609ea788d22bd59987cb8c297d80fe759b6dfdd422b2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ok3a8j9s/wheels/51/1e/3d/f92e27698ae7f6d586fd5482f569704ed8ba1a16f293bb0a5c
Successfully built nql


In [2]:
import random
import os

import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import schedules
from tensorflow.keras.preprocessing import text
from tensorflow.keras.models import Sequential

import nql

print(tf.__version__)

2.3.0


In [3]:
# Makes things as deterministic as possible for reproducibility
seed = 1234 #@param{type: "integer"}
tf.random.set_seed(seed)
random.seed(seed)

## Build the gridworld

In [4]:
class GridWorld():
  """A simple grid world for testing NQL algorithms.
  
  The world is square grid with every cell connected to its cardinal direction
  neighbors, with the exception of several holes which are inescapable."""
  size = None
  context = None
  words = ['go', 'top', 'left', 'right', 'bottom', 'center', 'up', 'down',
           'then']
  # A 2D dictionary mapping a cell and a valid move to another cell.
  # Ex: _cell_move_cell['cell_2_3']['up'] == 'cell_1_3'
  _cell_move_cell = {}
  # _cell_paths[start_cell][num_moves][end_cell] is a list of efficient paths
  # starting at start_cell, taking num_moves, and ending at end_cell
  _cell_paths = {}
  # _cell_dists[start_cell][dist] is a set of all of the cells exactly dist away
  # from start cell
  _cell_dists = {}
  
  def __init__(self, size, n_holes):
    if size ** 2 < n_holes:
      raise ValueError(
          'size ^ 2 (%d) must be great than n_holes (%d).'
          % (size ** 2, n_holes)) 
    self.size = size
    self.holes = set()
    
    # Remove cells
    while len(self.holes) < n_holes:
      r = random.randrange(size)
      c = random.randrange(size)
      self.holes.add((r,c))
    
    self.context = nql.NeuralQueryContext()
    self.context.declare_relation('up','place_t','place_t')
    self.context.declare_relation('down','place_t','place_t')
    self.context.declare_relation('left','place_t','place_t')
    self.context.declare_relation('right','place_t','place_t')
    kg_lines = []
    for (r,c) in [(r,c) for r in range(self.size) for c in range(self.size)]:
      if (r,c) in self.holes:
        continue
      self._cell_move_cell[self._cell(r,c)] = {}
      def connect_to_cell(dest_r, dest_c, dir):
        dest = self._cell(dest_r, dest_c)
        self._cell_move_cell[self._cell(r,c)][dir] = dest
        kg_lines.append('%s\t%s\t%s' % (dir, self._cell(r,c), dest))
      if(r > 0):
        connect_to_cell(r-1, c, 'up')
      if(r < self.size-1):
        connect_to_cell(r+1, c, 'down')
      if(c > 0):
        connect_to_cell(r, c-1, 'left')
      if(c < self.size-1):
        connect_to_cell(r, c+1, 'right')

    self.context.load_kg(lines=kg_lines, freeze=True)
    self.context.construct_relation_group('dir_g', 'place_t', 'place_t')

  def _cell(self, i, j):
    if (i, j) in self.holes:
      return 'hole_%d_%d' % (i+1, j+1)
    return 'cell_%d_%d' % (i+1, j+1)

  # Output: [(query, starting_cell, ending_cell, num moves)]
  def generate_examples(self, num, possible_moves=[1,2,3]):
    # Extends all paths from start_cell by 1 step
    def _extend_paths(start_cell):
      cell_paths = self._cell_paths[start_cell]
      cell_dists = self._cell_dists[start_cell]
      prev_path_len = len(cell_dists) - 1

      cell_dists.append(set())
      cell_paths.append({})
      if not len(cell_dists[prev_path_len]):
        # We're already longer than the longest path possible from this cell.
        return
  
      seen_cells = set(
          [cell for cells_at_dist in cell_dists for cell in cells_at_dist])
      for cell in cell_dists[prev_path_len]:
        prev_paths = cell_paths[prev_path_len][cell]
        if cell not in self._cell_move_cell:
          continue # This is a hole
        for (dir, next_cell) in self._cell_move_cell[cell].items():
          if next_cell not in seen_cells: # This is an efficient path
            if next_cell not in cell_dists[prev_path_len+1]:
              # This is the first time we found this cell
              cell_dists[prev_path_len+1].add(next_cell)
              cell_paths[prev_path_len+1][next_cell] = []
            for path in prev_paths:
              if prev_path_len == 0:
                new_path = 'go ' + dir
              else:
                new_path = path + ' then ' + dir
              cell_paths[prev_path_len+1][next_cell].append(new_path)
    def generate_example(num_moves):
      example = 'go '
      while True:
        start_row = random.randrange(self.size)
        start_col = random.randrange(self.size)
        starting_cell = self._cell(start_row, start_col)
        if not starting_cell in self._cell_move_cell:
          # Starting cell is a hole. Try again.
          continue
        if not starting_cell in self._cell_paths:
          # We've never started from this cell before.
          self._cell_paths[starting_cell] = [{starting_cell: ['']}]
          self._cell_dists[starting_cell] = [{starting_cell}]
        while len(self._cell_paths[starting_cell]) <= num_moves:
          _extend_paths(starting_cell)
        possible_ending_cells = list(self._cell_paths[starting_cell][num_moves])
        if not possible_ending_cells:
          # No paths num_moves long from this cell. Start over.
          continue
        ending_cell = random.choice(possible_ending_cells)
        path = random.choice(
            self._cell_paths[starting_cell][num_moves][ending_cell])
        return (path, starting_cell, ending_cell)

    examples = [None] * num
    while True:
      for i in range(num):
        # Each element of possible_moves is equally likely to be selected
        num_moves = random.choice(possible_moves)
        example = ''
        while not example:
          example, starting_cell, ending_cell = generate_example(num_moves)
        if examples[i] is None:
          examples[i] = ({
            "input_text": example,
            "start": tf.squeeze(self.context.one(starting_cell, 'place_t').tf),
            "start_name": starting_cell,
            "end_name": ending_cell,
            }, tf.squeeze(self.context.one(ending_cell, 'place_t').tf))
        yield examples[i]

In [5]:
grid_size =  10#@param {type: "integer"}
p_holes = 0.25 #@param {type: "number"}
grid_holes = grid_size ** 2 * p_holes
max_moves =  10#@param {type: "integer"}

env = GridWorld(grid_size, grid_holes)
print("Holes:", env.holes)

Holes: {(7, 3), (4, 7), (9, 1), (2, 1), (8, 9), (9, 0), (3, 3), (8, 1), (7, 6), (0, 4), (1, 1), (9, 7), (5, 4), (0, 0), (7, 1), (5, 2), (0, 5), (1, 9), (1, 0), (0, 8), (5, 3), (0, 1), (2, 0), (1, 8), (7, 8)}


In [6]:
from itertools import islice
print("Examples:", list(islice(env.generate_examples(4), 3)))
nqc = env.context

Examples: [({'input_text': 'go down then right', 'start': <tf.Tensor: shape=(99,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>, 'start_name': 'cell_9_4', 'end_name': 'cell_10_5'}, <tf.Tensor: shape=(99,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.,

In [7]:
dataset_size = 100000 #@param{type: "integer"}
train_gen = env.generate_examples(num=dataset_size, possible_moves=range(1, max_moves+1))
test_gen = env.generate_examples(int(dataset_size / 10))
max_seq_len = max(len(x["input_text"].split(" ")) for (x, _) in islice(train_gen, 1000))
print("max_seq_len is %d" % max_seq_len)
assert max_seq_len >= 2
assert max_seq_len < 50

max_seq_len is 20


In [8]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: (x for x in train_gen),
    output_types=(
        { "input_text": tf.string,
          "start": tf.float32,
          "start_name": tf.string,
          "end_name": tf.string,
        }, tf.float32),
    output_shapes=(
        { "input_text": tf.TensorShape([]),
          "start": tf.TensorShape([nqc.get_max_id('place_t')]),
          "start_name": tf.TensorShape([]),
          "end_name": tf.TensorShape([]),
        }, tf.TensorShape([nqc.get_max_id('place_t')])),
    )
text_dataset = train_dataset.map(lambda x,_: x["input_text"])

In [9]:
for i in train_dataset.take(2):
  print(repr(i))
for i in text_dataset.take(2):
  print(repr(i))
print(repr(train_dataset))
print(repr(train_dataset.element_spec))
print(repr(text_dataset))
print(repr(text_dataset.element_spec))

({'input_text': <tf.Tensor: shape=(), dtype=string, numpy=b'go down then down then down then right then down then right'>, 'start': <tf.Tensor: shape=(99,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>, 'start_name': <tf.Tensor: shape=(), dtype=string, numpy=b'cell_6_6'>, 'end_name': <tf.Tensor: shape=(), dtype=string, numpy=b'hole_10_8'>}, <tf.Tensor: shape=(99,), dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.,

## Example

In [10]:
name = nqc.get_entity_name(grid_size * 2, 'place_t')
print(name)
cell = nqc.one(name, 'place_t')
print(cell.eval())
cell = nqc.one(name, 'place_t').follow('right')
print(cell.eval())
cell = nqc.one(name, 'place_t').follow('right').follow('up')
print(cell.eval())

cell_3_6
{'cell_3_6': 1.0}
{'cell_3_7': 1.0}
{'cell_2_7': 1.0}


# Multiple Moves: Solve a problem with multiple possible templates

## RNN Model

In [11]:
#from tensorflow.compat.v2.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

class NqlFollowLayer(layers.Layer):
  def __init__(self, context, **kwargs):
    self.context = context
    super(NqlFollowLayer, self).__init__(**kwargs)
  
  def build(self, input_shape):
    assert isinstance(input_shape, list)
    super(NqlFollowLayer, self).build(input_shape)
  
  def call(self, x):
    assert isinstance(x, list)
    place_tf, dir_tf = x
    place_nql = self.context.as_nql(place_tf, "place_t")
    assert isinstance(place_nql, nql.NeuralQueryExpression)
    dir_nql = self.context.as_nql(dir_tf, "dir_g")
    assert isinstance(dir_nql, nql.NeuralQueryExpression)
    new_place_nql = place_nql.follow(dir_nql)
    return new_place_nql.tf
  
  def compute_output_shape(self, input_shape):
    return self.context.get_max_id("place_t")
  
  def get_config(self):
    config = super(NqlFollowLayer, self).get_config()
    config['context'] = self.context
    return config

def build_model(nqc, layer_width, embedding_dim, max_num_moves, text_dataset, num_layers=1, dropout=0.0, l1=0.0001, l2=0.0001):
  # Transforms text input into int sequences of length max_seq_len
  vectorize_layer = TextVectorization(max_tokens=len(env.words),
                                      output_mode="int",
                                      output_sequence_length=max_seq_len,
                                      name="VectorizationLayer",
                                      )
  vectorize_layer.adapt(text_dataset)

  # Build the encoder
  text_input = layers.Input(shape=(1,), dtype=tf.string, name="input_text")
  vec_input = vectorize_layer(text_input)
  # Account for the OOV token in this layer
  embedding_layer = layers.Embedding(input_dim=len(env.words)+1,
                                     output_dim=embedding_dim,
                                     mask_zero=True,
                                     embeddings_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                                     name="TextEmbedding")
  enc_emb = embedding_layer(vec_input)
  # Use an LSTM layer here to process the whole input sequence
  encoder_lstm = layers.LSTM(layer_width,
                             recurrent_dropout=dropout,
                             kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             recurrent_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             return_state=True,
                             name="EncoderLstm"
                             )
  enc_out, enc_state_h, enc_state_c = encoder_lstm(enc_emb)

  # Build the decoder
  decoder_lstm = layers.LSTM(layer_width,
                             recurrent_dropout=dropout,
                             kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             recurrent_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                             return_sequences=True,
                             name="DecoderLstm"
                             )

  decoder_move_model = Sequential(name="DecoderMoveModel")
  for i in range(num_layers-1):
    decoder_move_model.add(layers.Dense(layer_width,
                                        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                                        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                                        ))
  decoder_move_model.add(layers.Dense(
      nqc.get_max_id("dir_g"), activation="softmax"
  ))

  decoder_prob_model = Sequential(name="DecoderProbModel")
  for i in range(num_layers-1):
    decoder_prob_model.add(layers.Dense(layer_width,
                                        kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                                        bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
                                        ))
  decoder_prob_model.add(layers.Dense(1, activation="sigmoid"))

  initial_place = layers.Input(shape=(nqc.get_max_id("place_t"),), name="start")
  # Initial state of the decoder is the final state of the encoder
  dec_states = [enc_state_h, enc_state_c]
  cur_place = initial_place
  nql_layer = NqlFollowLayer(nqc, dynamic=True)
  # This should get turned into a tensor of the right shape the first time it's
  # updated.
  prob_remaining = 1.0
  final_place = tf.zeros(shape=(nqc.get_max_id('place_t'),))
  decoder_lstm_outs = decoder_lstm(
      tf.ones(shape=(1, max_num_moves, layer_width)),
      initial_state=dec_states)
  for i in range(max_num_moves):
    move_out = decoder_move_model(decoder_lstm_outs[:,i,:])
    prob_out = decoder_prob_model(decoder_lstm_outs[:,i,:])

    ## Use the output move to update the current place
    cur_place = nql_layer([cur_place, move_out])
    prob_stopping = prob_remaining * prob_out
    prob_remaining = prob_remaining - prob_stopping

    ## Update final output place
    final_place = final_place + (prob_stopping * cur_place)
  
  # Build the final model that goes from text to a place
  model = models.Model(inputs=[text_input, initial_place], outputs=final_place)
  return model

In [12]:
#@title Model Params { form-width: "25%" }
layer_width =  128 #@param{type: "integer"}
embedding_dim =   4#@param{type: "integer"}
num_layers =   3#@param{type: "integer"}
l1 = 0.00002 #@param{type: "number"}
l2 = 0.00002 #@param{type: "number"}
dropout = 0.1 #@param{type: "number"}

# For setting up the TextVectorization layer
text_dataset_sample = text_dataset.take(1024).batch(1024)
model = build_model(nqc,
                    layer_width=layer_width,
                    embedding_dim=embedding_dim,
                    max_num_moves=max_moves,
                    text_dataset=text_dataset_sample,
                    dropout=dropout,
                    l1=l1,
                    l2=l2,
                    num_layers=num_layers)

In [13]:
batch_size =  1024#@param{type: "integer"}
learning_rate = 0.0025 #@param{type: "number"}
clip_norm = 2.0 #@param{type: "number"}
decay = 0.005 #@param{type: "number"}

optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate,
                                      clipnorm=clip_norm,
                                      decay=decay,
                                     )

model.compile(optimizer=optimizer,
              loss="categorical_crossentropy",
              metrics=["accuracy"],
              # run_eagerly=True, # TODO(rofer): Figure out if this is only needed on CPUs
              )
batched_dataset = train_dataset.batch(batch_size)

callbacks = [
             keras.callbacks.TerminateOnNaN(),
]

In [14]:
n_epochs =  100#@param{type: "integer"}
history = model.fit(batched_dataset,
                    epochs=n_epochs,
                    steps_per_epoch=50,
                    callbacks=callbacks)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['start_name', 'end_name'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


50/50 [==============================] - 103s 2s/step - loss: 6.1354 - accuracy: 0.0300
Epoch 2/100
50/50 [==============================] - 101s 2s/step - loss: 4.6080 - accuracy: 0.0939
Epoch 3/100
50/50 [==============================] - 89s 2s/step - loss: 2.7759 - accuracy: 0.2668
Epoch 4/100
50/50 [==============================] - 89s 2s/step - loss: 2.1506 - accuracy: 0.3858
Epoch 5/100
50/50 [==============================] - 89s 2s/step - loss: 1.8828 - accuracy: 0.4398
Epoch 6/100
50/50 [==============================] - 90s 2s/step - loss: 1.6598 - accuracy: 0.4837
Epoch 7/100
50/50 [==============================] - 94s 2s/step - loss: 1.4606 - accuracy: 0.5353
Epoch 8/100
50/50 [==============================] - 92s 2s/step - loss: 1.3069 - accuracy: 0.5760
Epoch 9/100
50/50 [==============================] - 92s 2s/step - loss: 1.1738 - accuracy: 0.6135
Epoch 10/100
50/50 [==============================] - 93s 2s/step - loss: 1.0895 - accuracy: 0.6317
Epoch 11/100
50/50 

## Evaluation

## Accuracy by # of moves

In [15]:
# Accuracy by number of moves
num_batches = 3 #@param{type: "integer"}
num_samples = num_batches * batch_size
for moves in range(1, max_moves + 1):
  print("For %d moves:" % moves)
  one_move_dataset = tf.data.Dataset.from_generator(
    lambda: (x for x in env.generate_examples(
      num=num_samples, possible_moves=[moves]
    )),
    output_types=(
        { "input_text": tf.string,
          "start": tf.float32,
          "start_name": tf.string,
          "end_name": tf.string,
        }, tf.float32),
    output_shapes=(
        { "input_text": tf.TensorShape([]),
          "start": tf.TensorShape([nqc.get_max_id('place_t')]),
          "start_name": tf.TensorShape([]),
          "end_name": tf.TensorShape([]),
        }, tf.TensorShape([nqc.get_max_id('place_t')])),
    )
  model.evaluate(one_move_dataset.take(num_samples).batch(batch_size))
  print("")


For 1 moves:


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['start_name', 'end_name'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


3/3 [==============================] - 2s 722ms/step - loss: 0.1057 - accuracy: 1.0000

For 2 moves:
3/3 [==============================] - 2s 746ms/step - loss: 0.1099 - accuracy: 1.0000

For 3 moves:
3/3 [==============================] - 2s 731ms/step - loss: 0.1938 - accuracy: 0.9583

For 4 moves:
3/3 [==============================] - 2s 730ms/step - loss: 0.2202 - accuracy: 0.9574

For 5 moves:
3/3 [==============================] - 2s 732ms/step - loss: 0.2401 - accuracy: 0.9544

For 6 moves:
3/3 [==============================] - 2s 723ms/step - loss: 0.3043 - accuracy: 0.9287

For 7 moves:
3/3 [==============================] - 2s 749ms/step - loss: 0.3290 - accuracy: 0.9160

For 8 moves:
3/3 [==============================] - 2s 759ms/step - loss: 0.3472 - accuracy: 0.9030

For 9 moves:
3/3 [==============================] - 2s 737ms/step - loss: 0.3586 - accuracy: 0.8952

For 10 moves:
3/3 [==============================] - 2s 730ms/step - loss: 0.3465 - accuracy: 0.8975



## Live Evaluation

In [16]:
query = "go right then down" #@param{type: "string"}
start = "cell_3_3" #@param{type: "string"}

nql_start = nqc.one(start, "place_t")
if "<UNK>" in nql_start.eval():
  print("%s is not a valid starting point. It's probably a hole or a typo." %
        start)
else:
  example_dataset = tf.data.Dataset.from_tensors({
      "input_text": query,
      "start": tf.squeeze(nql_start.tf),
  })
  pred = model.predict(example_dataset.batch(1))
  print(pred.shape)
  pred_nql = nqc.as_nql(pred, "place_t")
  print(pred_nql.eval(as_top=3))

(1, 99)
[('hole_4_4', 0.9885572), ('cell_3_5', 0.0068089347), ('cell_5_3', 0.0046146773)]
